In [4]:
import pandas as pd
import os

## Load dataset

In [13]:
file_path = "Sample - Superstore.csv"
df = pd.read_csv(file_path, parse_dates=["Order Date", "Ship Date"], encoding="cp1252")

In [14]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


## Clean column names to avoid issues like KeyError due to invisible characters

In [20]:
# Clean column names to avoid issues like KeyError due to invisible characters
df.columns = df.columns.str.strip()  # Remove leading/trailing whitespaces
df.columns = df.columns.str.replace('\xa0', ' ', regex=False)  # Replace non-breaking spaces with normal spaces
df.columns = df.columns.str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with a single space
df.columns = df.columns.str.replace('\n', '', regex=False)  # Remove newline characters


In [21]:
print(df.columns.tolist())

['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']


## Define the key categorical columns to analyze

In [22]:
key_columns = ["Ship Mode", "Segment", "Country", "City", "State", "Region", "Category", "Sub-Category"]

In [23]:
# Create a dictionary with number of unique values per column
unique_summary = {col: df[col].nunique() for col in key_columns}

In [24]:
# Display the summary in a readable table
pd.DataFrame(list(unique_summary.items()), columns=["Column", "Unique Values"])

,Column,Unique Values
0,Ship Mode,4
1,Segment,3
2,Country,1
3,City,531
4,State,49
5,Region,4
6,Category,3
7,Sub-Category,17


In [26]:
# Missing Values Analysis
# Count and percentage of missing values per column

missing_report = df.isnull().sum()
missing_percent = (missing_report / len(df)) * 100

missing_df = pd.DataFrame({
    "Missing Values": missing_report,
    "Missing %": missing_percent.round(2)
}).sort_values(by="Missing Values", ascending=False)

print("Missing values report:")
display(missing_df)

Missing values report:


,Missing Values,Missing %
Row ID,0,0.0
Postal Code,0,0.0
Discount,0,0.0
Quantity,0,0.0
Sales,0,0.0
Product Name,0,0.0
Sub-Category,0,0.0
Category,0,0.0
Product ID,0,0.0
Region,0,0.0


In [27]:
# Distinct values in key categorical columns
# This helps to understand what unique categories exist per feature

columns_to_check = ["Ship Mode", "Segment", "Country", "Region", "Category", "Sub-Category"]

distinct_values = {
    col: df[col].dropna().sort_values().unique().tolist()
    for col in columns_to_check
}

In [29]:

# Aligning value lists for display as DataFrame
max_len = max(len(vals) for vals in distinct_values.values())
aligned_data = {
    col: vals + [None] * (max_len - len(vals))
    for col, vals in distinct_values.items()
}

distinct_df = pd.DataFrame(aligned_data)

print("Distinct categorical values:")
display(distinct_df)

Distinct categorical values:


,Ship Mode,Segment,Country,Region,Category,Sub-Category
0,First Class,Consumer,United States,Central,Furniture,Accessories
1,Same Day,Corporate,None,East,Office Supplies,Appliances
2,Second Class,Home Office,None,South,Technology,Art
3,Standard Class,None,None,West,None,Binders
4,None,None,None,None,None,Bookcases
5,None,None,None,None,None,Chairs
6,None,None,None,None,None,Copiers
7,None,None,None,None,None,Envelopes
8,None,None,None,None,None,Fasteners
9,None,None,None,None,None,Furnishings


In [34]:
#  Drop unnecessary columns
# We remove columns that are not useful for analysis or will not be part of our data model
if "Row ID" in df.columns:
    df.drop(columns=["Row ID"], inplace=True)
    print("'Row ID' column dropped.")
else:
    print("'Row ID' column was already missing.")

'Row ID' column was already missing.


In [35]:
# Drop rows with critical missing values
# We remove rows where key identifiers are missing
df.dropna(subset=["Order Date", "Ship Date", "Customer ID", "Product ID"], inplace=True)
print("Dropped rows with missing key identifiers.")

Dropped rows with missing key identifiers.


In [36]:
# Fill non-critical missing values
# For Postal Code (if missing), we fill with 0 or placeholder value
if "Postal Code" in df.columns:
    df["Postal Code"].fillna(0, inplace=True)
    print("Filled missing postal codes with 0.")

Filled missing postal codes with 0.


In [37]:
# Create a new column for delivery delay in days
df["Delivery Delay"] = (df["Ship Date"] - df["Order Date"]).dt.days
print("Created 'Delivery Delay' column.")

Created 'Delivery Delay' column.


In [53]:
# Customer Dimension
dim_customer = df[["Customer ID", "Customer Name", "Segment"]].drop_duplicates().reset_index(drop=True)

In [54]:
# Product Dimension
dim_product = df[["Product ID", "Product Name", "Category", "Sub-Category"]].drop_duplicates().reset_index(drop=True)

In [55]:
# Region Dimension
dim_region = df[["Country", "State", "Region", "City", "Postal Code"]].drop_duplicates().reset_index(drop=True)
dim_region["Region ID"] = dim_region.index + 1  # Surrogate key

In [56]:
# Transport Dimension
dim_transport = df[["Ship Mode"]].drop_duplicates().reset_index(drop=True)
dim_transport["Transport ID"] = dim_transport.index + 1  # Surrogate key

In [57]:
# Date Dimension (based on Order Date)
dim_date = pd.DataFrame()
dim_date["Order Date"] = pd.to_datetime(df["Order Date"].unique())
dim_date["Date Key"] = dim_date["Order Date"].dt.strftime("%Y%m%d").astype(int)
dim_date["Year"] = dim_date["Order Date"].dt.year
dim_date["Month"] = dim_date["Order Date"].dt.month
dim_date["Quarter"] = dim_date["Order Date"].dt.quarter
dim_date["Weekday"] = dim_date["Order Date"].dt.day_name()

In [58]:
# Merge Region ID
df = df.merge(dim_region, on=["Country", "State", "Region", "City", "Postal Code"], how="left")


In [59]:
# Merge Transport ID
df = df.merge(dim_transport, on="Ship Mode", how="left")

In [60]:
# Create Date Key
df["Date Key"] = df["Order Date"].dt.strftime("%Y%m%d").astype(int)

In [61]:
# Final Fact Table
fact_sales = df[[
    "Order ID", "Customer ID", "Product ID",
    "Region ID", "Transport ID", "Date Key",
    "Quantity", "Sales", "Discount", "Profit", "Delivery Delay"
]]

In [62]:
# Save all dimension and fact tables
dim_customer.to_csv("dim_customer.csv", index=False)
dim_product.to_csv("dim_product.csv", index=False)
dim_region.to_csv("dim_region.csv", index=False)
dim_transport.to_csv("dim_transport.csv", index=False)
dim_date.to_csv("dim_date.csv", index=False)
fact_sales.to_csv("fact_sales.csv", index=False)